In [2]:
#!/usr/bin/env python3
# coding=utf-8
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback
from scipy import stats
from itertools import product as product

pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

g_worthDir = '../pkl'  # 
g_parentDir = '../../..'  # 
g_dbfile = f"{g_parentDir}/pkl/hq.db" # 
g_name = g_access = 'logic'

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

# df3 =pd.DataFrame(np.random.randn(8,4), columns=list('ABCD') )

# print(f'{os.path.basename(__file__)}:{sys._getframe().f_code.co_name}:{sys._getframe().f_lineno}')
print(f"{sys.version} , {sys.executable} ")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")

3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0] , /home/at/bin/anaconda3/bin/python 
2023-03-15 17:31:40 *****import**v1.0.1*** 


In [ ]:
iniTime = datetime.now()
time.sleep(10) # 60*60  3*60
nowTime = datetime.now()
diffTime = (nowTime - iniTime)

print(f"{diffTime = },  {diffTime.seconds = },  {diffTime.min = },  {diffTime.days = },  {diffTime.max = }")


In [ ]:
curSec = int(time.strftime("%s"))
curSec2 = int(time.time())
print(f"{curSec = } , {curSec2 = } ")

In [ ]:
print(dir(diffTime))
print(f"{diffTime = },  {diffTime.seconds = },  {diffTime.min = },  {diffTime.days = },  {diffTime.max = }")

In [ ]:
d={'x':[0,0,1,2,1,0,0,4,4,0,0]}
df=pd.DataFrame(d)

print(f"{df = } \n")

i=0
x=df['x'][i]


while x==0:
    del df['x'][i]
    i+=1
    x=df['x'][i]
df=df[i:] 

print(f"{df = }")


In [15]:
data={'Process_ID':['12345-98', '23547-75', '85763-99','44231-56','78456-00','53218-87'],
     'Date': ['2021-06-30','2022-08-10','2023-02-15','2023-10-02','2024-04-03','2021-06-25']}

df=pd.DataFrame(data)

df['Date']=pd.to_datetime(df['Date'])

print(df)

  Process_ID       Date
0   12345-98 2021-06-30
1   23547-75 2022-08-10
2   85763-99 2023-02-15
3   44231-56 2023-10-02
4   78456-00 2024-04-03
5   53218-87 2021-06-25


In [16]:
# df['new'] = df['Date'].apply(lambda x: True if 
#     (x - pd.Timestamp.now() < pd.Timedelta('30 days')) 
#     and x > pd.Timestamp.now() else False)

df['new'] = df['Date'].apply(lambda x: True if ((pd.Timestamp.now() - x) < pd.Timedelta('30 days') and pd.Timestamp.now() > x )  else False)

print(df)

  Process_ID       Date    new
0   12345-98 2021-06-30  False
1   23547-75 2022-08-10  False
2   85763-99 2023-02-15   True
3   44231-56 2023-10-02  False
4   78456-00 2024-04-03  False
5   53218-87 2021-06-25  False


In [10]:
print(f"{pd.Timestamp.now() = } , {pd.Timedelta('30 days')}")
print( pd.Timedelta('30days') )

pd.Timestamp.now() = Timestamp('2023-03-15 09:38:33.660427') , 30 days 00:00:00
30 days 00:00:00


In [1]:
# notebook pwd
from notebook.auth import passwd # 
passwd()

'argon2:$argon2id$v=19$m=10240,t=10,p=8$q1VncAWCktr946a4S9PqfA$inDVF7tDFAIdptOh2VAcXHM8xOWQ8V0CLfjhszyYJ8w'

In [ ]:
# print(os.getcwd()) 

In [ ]:
# df_trade.to_pickle(f'{g_dir}/tradedf.pkl' )
df_tra = pd.read_pickle(f"/workspaces/jupyter/tmp/tradedf.pkl" )

print(f"{df_tra.shape}")
# print(f"{df_tra}")
display(df_tra)

In [ ]:
rt_df = pd.DataFrame([{'asset': 2, 'update': time.strftime('%Y%m%d %H:%M:%S')}])
# print(type(rt_df), len(rt_df) )
# print(f" {rt_df.info}" )
# display(rt_df)
print(f"rt_df :\n{rt_df}")

print(f"{rt_df.columns}")
print(f"{rt_df.index}")


In [ ]:
dict = {'a':'apple','b':'banana'}
df = pd.DataFrame([dict])

print(f"df :\n{df.b}")

# print(f"{df.columns}")
# print(f"{df.index}")

In [ ]:
df2 = pd.DataFrame.from_dict({
 'apple': 3,
 'banana': 5,
 'mango': 7,
 'apricot': 1,
 'kiwi': 8,
 'orange': 3}, orient='index').rename(columns={0:'Qunatity'})

print(f"df2 :\n{df2}")

# print(f"{df2.columns}")
# print(f"{df2.index}")

In [ ]:
# df3 = pd.DataFrame(np.arange(40).reshape(10,4),index=pd.date_range('2022-01-01',periods=10),columns=list('ABCD'))
df3 =pd.DataFrame(np.random.randn(8,4), columns=list('ABCD') )

print(f"df3 :\n{df3.A}")

# print(f"{df3.columns}")
# print(f"{df3.index}")


In [ ]:
# package
import os

try:
    import pandas as pd
    import time
except:
    # !pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn

    os.system('pip install pandas numpy matplotlib scipy tqdm pymysql sqlalchemy ')
    os.system('pip install requests_toolbelt flask pandas_datareader sshtunnel torch ')
    os.system('pip install pymssql EMD-signal websocket-client ccxt statsmodels sklearn')
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
rng = np.random.default_rng()
x = rng.random(10)
y = 1.6*x + rng.random(10)
res = stats.linregress(x, y) # 线性拟合

print(f"res: {res}")

# LinregressResult(斜率slope=1.4609737282140889, 截距intercept=0.6260166856203904, 皮尔逊相关系数rvalue=0.76760585460041 rvalue 的平方等于决定系数, 对零假设为斜率为零的假设检验的p值pvalue=0.009535323744696659, 估计的斜率(梯度)的标准误差stderr=0.43128508714183683, 估计的截距的标准误差intercept_stderr=0.2307834268202402)

print(f"拟合度 R-squared: {res.rvalue**2:.6f}")

# ret = (result_[0]**2)*(result_[2]**2)
print(f"(res[0]**2)*(res[2]**2): {(res[0])} **2 * {(res[2])} **2 = {(res[0]**2)*(res[2]**2)}")


In [ ]:
df = pd.DataFrame()
# df['index'] = range(0, 4059)
df['index'] = np.nan

print(f"df :\n{df}")
# print(f"range(0, 4059) :\n{range(0, 4059)}")


In [ ]:
reg_wid = 10 + (np.arange(10)) * 5

stat_wid = 60 + (np.arange(10)) * 30

threshold = 0.5 + (np.arange(20)) * 0.1

print(f"reg_wid :\n{reg_wid}")
print(f"stat_wid :\n{stat_wid}")
print(f"threshold:\n{threshold}")

df_para = pd.DataFrame(list(product(reg_wid, stat_wid,threshold)))

print(f"df_para:\n{df_para}") # 10*10*20  :  10   60  0.5

print(f"df_para.iloc[621] : \n{df_para.iloc[621]}")  # 25  90  0.6



In [ ]:
import talib
# import mpl_finance as mpf
import mplfinance as mpf
import warnings
import akshare as ak

In [ ]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20210301", end_date='20210907', adjust="qfq")
print(stock_zh_a_hist_df)

In [ ]:
stock_zh_a_daily_qfq_df = ak.stock_zh_a_daily(symbol="sz000002", start_date="20221003", end_date="20221106", adjust="qfq")
print(stock_zh_a_daily_qfq_df)

In [ ]:
import sys

print(sys.version)
print(sys.executable)

In [ ]:
import sklearn
print("sklearn :" , sklearn.__version__)

from sklearn.model_selection import train_test_split

from sklearn import model_selection

In [ ]:
from datetime import datetime, timedelta
# LOCAL_TIMEZONE = datetime.datetime.now().astimezone().tzinfo  #datetime.timezone.utc
print(f"{datetime.now().astimezone().tzinfo}")
print(f"{datetime.utcnow().astimezone().utcoffset().total_seconds()}")
print(f"{28800 - datetime.utcnow().astimezone().utcoffset().total_seconds()}")

secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"
print(f"{secret_key[:4]}")

serverUrl = 'http://testnet.binancefuture.com'
serverUrl = 'http://www.binance.com'

print(f"{serverUrl.split('.')[1] }")


In [ ]:
from datetime import datetime, timedelta
import time

n1 = datetime.now()
print(n1, n1.strftime("%Y%m%d %H:%M:%S %s %Z "), datetime.now().strftime("%Y%m%d %H:%M:%S %s %Z "))  # datetime.now().strftime("%Y%m%d")  20230110 04:05:01 1673323501
print(time.mktime(n1.timetuple())) # 1673323501.0

print(f"{28800 - datetime.utcnow().astimezone().utcoffset().total_seconds()}") #时差 28800.0

print(0, datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%Y%m%d %H:%M:%S %s %Z ")  ) # 0 20230110 00:00:00 1673308800 

print(int(datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%s")) ) # 1673308800

print(1, int(datetime.strptime("20220914", "%Y%m%d").strftime("%s")) ) # 1 1663113600
print(2, (datetime.strptime("20220914", "%Y%m%d") + timedelta(days=2) ).strftime("%s") )  # 2 1663286400
print(3,  ( datetime.fromtimestamp(1663113600)+ timedelta(days=3) ).strftime("%s")   ) #  3 1663372800

print(4, datetime.fromtimestamp(1675491616)) # 4 2022-09-14 00:09:59
print(5, datetime.fromtimestamp(1675786281)) #startSec=1663027200, endSec=1663113600,  # 5 2022-09-14 00:00:00

print(6, datetime.fromtimestamp(1656345600999//1000 +28800 )) # 6 2022-06-28 00:00:00
print(7, datetime.fromtimestamp(1664312400000//1000 +28800))   # 7 2022-09-28 05:00:00
print(8, datetime.fromtimestamp(1658975892471//1000))   # 8 2022-07-28 02:38:12
print(9, datetime.fromtimestamp(1667810456119//1000))   # 9 2022-11-07 08:40:56 

print(10, (datetime.now()- timedelta(days=10) ).strftime("%s000") )  # 10 1672459501000


In [ ]:
# print(f'{ int(time.time() ) }')

curDateTime = datetime.fromtimestamp( 1663113600//1000 +28800).strftime('%Y%m%d %H%M%S %w')

hhmm = curDateTime[9:13]
weekday = curDateTime[-1:]

print(f"{curDateTime =} , {hhmm =} , {weekday =} , ")


g_name = 'similarity'
print(f"{g_name[:2].upper() }")


In [ ]:
def fun_upvar(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df['pct'][idx])
    ret = np.std(dfs['pct'][dfs['pct']>0])
    return ret

In [ ]:
def fun_upvol(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df.iloc[idx])
    ret = np.sum(dfs['amount'][dfs['pct']>0])/np.sum(dfs['amount'])
    return ret
def fun_downvar(idx, df):
    idx = [int(i) for i in idx]
    dfs=pd.DataFrame(df['pct'][idx])
    ret = np.std(dfs['pct'][dfs['pct']<0])
    return ret

In [ ]:
###%pip package###whl###module###
try:
    import pandas as pd
    import numpy as np
    import time
    from datetime import datetime, timedelta
except:
    %pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn sshtunnel
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')    

In [ ]:
# !pip install -r requirements.txt

%pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn pycryptodome jwt werkzeug  

In [ ]:
%pip install pycryptodome
# !cal

In [ ]:
df = pd.DataFrame(np.arange(40).reshape(10,4),index=pd.date_range('2022-01-01',periods=10),columns=list('ABCD'))
df =pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'])
df =pd.DataFrame(np.array([i for i in range(1,41)]).reshape(10,4), columns=['A','B','C','D'])

df[ df["D"] > 1000  ].index.tolist()[-1] = 47
df.iloc[-1].index.tolist() = ['A', 'B', 'C', 'D', 'index', 'idx']

In [ ]:
df =pd.DataFrame(np.array([i for i in range(1,1153)]).reshape(288,4), columns=['A','B','C','D'])
print(f'{df}')

print( df[ df["D"] > 1000  ].index.tolist()) # = 47
print( df[ df["D"] > 1000  ].index.tolist()[-1]) # = 47
print( df[ df["D"] > 2000  ].index.tolist().empty() ) # = 47

print( len(df.index) - df[df['D'] > 2000 ].index.tolist()[-1]  )
# (len(df_panel.index) - df_panel[df_panel['factor'] > float(thd) ].index.tolist()[-1] == wid2 -1)

In [ ]:

list_2 = [] # empty  empty
if list_2:
    print('list_2 is none')

In [ ]:
df =pd.DataFrame(np.array([i for i in range(-10,30)]).reshape(10,4), columns=['A','B','C','D'])

# print(df)
df['A0'] = df.A.shift(0)
df['A1'] = df.A.shift(1)  # +下移
df['A_1'] = df.A.shift(-1) # -上移
df['C_sign'] = np.sign(df['C'])
df['E'] = np.nan

print(f"{ type(df['E'].iloc[-1])} , {df['E'].iloc[-1]}")
if not np.isnan(df['E'].iloc[-1]):
    print(1)
else:
    print(2)


# print(f"{len(df)}")
print(df)

curSign = 0 if np.isnan(df['A_1'].iloc[-1]) else int(df['A_1'].iloc[-1])
print(f"{df['A_1'].iloc[-1] =} , {curSign = }")

curSign = 0 if np.isnan(df['A_1'].iloc[-2]) else int(df['A_1'].iloc[-2])
print(f"{df['A_1'].iloc[-2] =} , {curSign = }")

In [ ]:
totalWorthdf2 = pd.read_pickle('/workspaces/jupyter/tmp/totalWorthdf2.pkl') 
# ukdf.to_pickle(f"{g_worthDir}/uk4hdf.pkl" )


In [ ]:
print(totalWorthdf2.shape)
print(totalWorthdf2.head(2))
print(totalWorthdf2.tail(2))


In [ ]:
Worth_df = totalWorthdf2[['dateTime' , 'totalworth' , 'time' ]]
print(Worth_df.shape)
print(Worth_df.head(2))
print(Worth_df.tail(2))
Worth_df.to_csv('/workspaces/jupyter/tmp/similarity2_5min.csv')


In [ ]:
# simi_df = totalWorthdf2[totalWorthdf2['name']=='testStrategy']
simi_df = totalWorthdf2[['date' , 'totalworth' , 'time' ]].groupby('date').last()
print(simi_df.shape)
print(simi_df.head(2))
print(simi_df.tail(2))
simi_df.to_csv('/workspaces/jupyter/tmp/similarity2_day.csv')